In [103]:
# TODOs: Operate DB Class
# TODOs: Use SQLite
# TODOs: Implement BM25

import sys
from pathlib import Path
sys.path[0] = str(Path(sys.path[0]).parent)

from utils.elastic_db import ElasticDB

# INIT DB OBJECT
PORT = "http://localhost:9200"
INDEX_NAME = "news_cc"

news_db = ElasticDB(elastic_port=PORT, elastic_index=INDEX_NAME)
wiki_db = ElasticDB(elastic_port=PORT, elastic_index="knowledge")

INFO:utils.elastic_db:Connecting to http://localhost:9200 
INFO:utils.elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 
INFO:utils.elastic_db:Connecting to http://localhost:9200 
INFO:utils.elastic_db:Connected to <Elasticsearch(['http://localhost:9200'])> 


In [104]:
### LOAD DATASETS ###
import json

data = [json.loads(ln) for ln in open("../../data/train_cmv_cleaned.jsonl")]
topics = [json.loads(ln) for ln in open("../../data/claim_topics.jsonl")]

ex_retrieval = [json.loads(ln) for ln in open("../../data/wiki_doc_retrieved_from_op_train.jsonlist")]
ex_ranked = [json.loads(ln) for ln in open("../../data/selected_evidence.jsonl")]

conan = [json.loads(ln) for ln in open("../../data/CONAN.json")]

In [105]:
len(data)

3456

In [106]:
### SAMPLE OUTPUT ###
import random

# TODOs: Replicate Output, Passage Evidence Retrieval
# TODOs: Replicate Output, Passage Ranking

_ = random.randint(0, 1000)
ranked = ex_ranked[_]
retireval = ex_retrieval[_]

retireval

{'tid': 't3_5c8txh',
 'retrieval_results': [{'retrieved_document_titles': ['François Bigot',
    'Human cytomegalovirus',
    'Congenital cytomegalovirus infection',
    'Trophime Bigot',
    'Cucumber mosaic virus'],
   'query': 'cmv a xenophobic bigot a xenophobic bigot Knowingly',
   'sentence': 'CMV : Knowingly voting for a xenophobic bigot makes you a xenophobic bigot yourself . '},
  {'retrieved_document_titles': ['Donald Trump presidential campaign, 2016',
    'Donald Trump',
    'Political positions of Donald Trump',
    'International reactions to the United States presidential election, 2016',
    'United States presidential election, 2016'],
   'query': 'trump premise sexist racist',
   'sentence': 'Premise 1 : Trump is sexist , racist and bigoted in general . '},
  {'retrieved_document_titles': ['Political positions of Donald Trump',
    'Donald Trump presidential campaign, 2016',
    'Theodor Plievier',
    'Donald Trump',
    'Legal affairs of Donald Trump'],
   'query':

In [37]:
# TODOs: News Data
# TODOs: Ranking
# TODOs: Ranking, Cosine
# TODOs: Research Evidence Retireval: Context Aware, Neural Retrieval
# TODOs: Stance
# TODOs: Target ADUs: Premises, Claims, discard non-ADUs, thus reducing noise over retreival 

# TODOs: Paralellise

In [129]:
from utils.keyphrase_extraction import extract_keyphrase
# from keyphrase_vectorizers import KeyphraseCountVectorizer

import random
import re

_ = random.randint(0, 1000)
print(_)

arg = data[_]["arguments"]
print(arg)

# def to_sentences(text):
#     sents = [i for i in re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)]
#     return sents

# sentences = to_sentences(arg)
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', arg)

len(sentences)

queries = []
results = []

for sent in sentences:
    print(sent)

    if len(sent) < 5:
        continue

    kp = extract_keyphrase(sent, n_kp=3)
    query = ", ".join(i for i in kp)
    
    # TODOs: Re-init DB with smaller passage size
    titles = [i["_source"]["document"]["title"] for i in news_db.search(query_=query, k=10)]
    evidence = [i["_source"]["document"]["text"] for i in news_db.search(query_=query, k=10)]

    results.append({
        "argument_sentence": sent, 
        "retrieved_documents_titles": titles,
        "query": query,
        "retrieved_evidence": evidence
    })

90
Recently in the news the C.I.A. and Bush administration have been under a lot of heat for torturing prisoners. In the grand scheme of things what are a few lives compared to the safety of thousands of others. The people they are torturing are a global threat its not out of patriotism that Im trying to justify this for the sake of protecting good old murica. I think the main argument against me would be that theres a chance that some of these people may be innocent. But if someone came up to me and said three other people and I need to be tortured because only one of us is a terrorist and its either the four of us or potentially thousands of others I would volunteer in a heartbeat. I am already well aware the government spends billions of dollars on military related things that the public has no idea about and Im okay with that because I understand that those are keeping us safe.I think my overall opinion is that its good to be a good person and cooperate with people but you also hav

INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.057s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.005s]


In the grand scheme of things what are a few lives compared to the safety of thousands of others.


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.125s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.011s]


The people they are torturing are a global threat its not out of patriotism that Im trying to justify this for the sake of protecting good old murica.


...

INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.078s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.008s]


But if someone came up to me and said three other people and I need to be tortured because only one of us is a terrorist and its either the four of us or potentially thousands of others I would volunteer in a heartbeat.


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.059s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.015s]


I am already well aware the government spends billions of dollars on military related things that the public has no idea about and Im okay with that because I understand that those are keeping us safe.I think my overall opinion is that its good to be a good person and cooperate with people but you also have to understand there are people who you cant negotiate with and you just have to kill or torture them.


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.090s]
INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.015s]


In [158]:
### MERGE ###

# Results == a list of counter-evidence articles per sentence

merged = []
for counter_ev in results:
    sentence = counter_ev["argument_sentence"]
    evidence = counter_ev["retrieved_evidence"]
    sent_kp = counter_ev["query"]

    merged_ev = ", ".join(retrieved for retrieved in evidence)

    merged.append((sentence, sent_kp, merged_ev))

merged

[('Recently in the news the C.I.A. and Bush administration have been under a lot of heat for torturing prisoners.',
  'prisoners, bush administration, heat',
  'Bush had just been elected after a long run as Ronald Reagan’s vice president. For the most part, I listened very little to what the Bush administration had to say in those days. I confess I wasn’t a supporter of his outlook or his economic priorities, which didn’t seem to match with the abject poverty I saw around me at the time in then-rural West Florida and the poor governance in environmental safety. But that afternoon, dogged by the sweltering heat of a humid Florida drought, this particular news bite caught my attention: Americans’ wages were too high. They were unsustainable in the current economy., Bush administration did not reinstate them. During the Obama administration, opponents of Planned Parenthood funding sought legislation to cut it off. Their tactic made sense given the liberalism of the administration. Execut

In [157]:
### PASSAGE RANKING : KEYWORD OVERLAP ###

test = merged[2]
test
sent, kp, merge = test

sent, sent_kp

6

In [172]:
merged_sentences = to_sentences(merge)
print(len(merged_sentences))
print(sentence)
# print(sent_kp)
print("")

# TODOs: Improve Scoring Function for whole vs partial keyphrase match
test = ["overall", "some opinion", "distinction", "elevator"]
#print(sent_kp.lower().split())

def overlap_score(evidence_kp, adu_kp):
    score = 0
    # Split Keyphrase into components, scoring partial units as overlap
    for i in evidence_kp:
        for j in i.split():
            if adu_kp.find(j) != -1: score += 1
            
            else: continue
    
    return score

# TODOs: Speed-up, Parrelleise, Yield
sentence_scores = []
for sent, sent_kp, evidence in merged[0:1]:
    sent_overlap_score = 0
    
    evidence_sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', evidence)

    for ev_sent in evidence_sentences:
        evidence_kp = extract_keyphrase(ev_sent)
        sent_overlap_score += overlap_score(evidence_kp, sent_kp)
        
        sentence_scores.append((ev_sent, sent_overlap_score))
    

sent_overlap_score


40
I am already well aware the government spends billions of dollars on military related things that the public has no idea about and Im okay with that because I understand that those are keeping us safe.I think my overall opinion is that its good to be a good person and cooperate with people but you also have to understand there are people who you cant negotiate with and you just have to kill or torture them.



41

In [173]:
sentence_scores
sentence_scores.sort(key=lambda y: y[1], reverse=True)

sentence_scores

[('Bush oversaw years of brutal torture and war crimes, the detention of hundreds of Muslims, and inaugurated the covert Controlled Application Review and Resolution Program (CAARP), which led the barring of Muslims from immigration benefits and their placement on a “Terror Watch List.” Nevertheless, Bush has managed to rebrand himself into Bush the Painter, the endearing grandfather figure.',
  41),
 ('Yet, in every instance that Trump or members of his administration target Muslims, the reanimated corpse of the Bush presidency is forced into the public arena again, and a parade of commentators tell a story that is pure fiction about an administration that only exists in their heads.',
  40),
 ('What is it about this short address to a congregation of Muslims in Washington, D.C., that allows one to completely inoculate themselves against an administration responsible for horrific crimes against Muslims?',
  40),
 ('It’s almost incomprehensible how one could look back with longing at a

In [ ]:
# print(kp)

# def count(string, substring):
#     n = len(substring)
#     cnt = 0
#     for i in range(len(string) - n):
#         if string[i:i+n] == substring:
#             cnt += 1
#     return cnt

# def hasIntersection(a, b):
#         score = 0
#         a = set(a) 
#         b = set(b)
#         if a.intersection(b):
#             score += 1

#         return score

# def overlap_score(count_kp):
#     score = 0

#     for i in count_kp:
#         score += count(sent_kps, count_kp)

#     return score

# scores = []
# for sent in merged_sentences:
#     if sent:
#         # TODOs: Try-Catch, Handel Errors
#         kp = [i for i in extract_keyphrase(sent, n_kp=3)]
#         scores.append((kp, overlap_score(kp)))

# print(sent_kps)
# scores

In [59]:
print(sent_kp)

view, arguments, logic, line


In [ ]:
### ATTACKING PEMISES ###

# from BERT_adu_classifier import predict

# premises = []
# for sent in sentences:
#     prediction = predict(sent)
    
#     if prediction == "premise":
#         premises.append(sent)

In [26]:
# # Argument Level

# results = []

# _ = 90
# hate_ = sample_hate[0]["conan"][_]["hateSpeech"]

# # queries = []
# # sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', hate_)

# results = []

# # TODOs: Refactor key_BERT as Class for multiple oututs
# #kp_ = (exctract_keyphrase(hate_, n_gram=1, n_kp=1), exctract_keyphrase(hate_, n_gram=2, n_kp=1), exctract_keyphrase(hate_, n_gram=3, n_kp=1))

# kp_ = (exctract_keyphrase(hate_, n_gram=5, n_kp=1))

# query = " ,".join(i for i in kp_)
# query

# res = [i["_source"]["document"]["title"] for i in news_db.search(query_=query, k=5)]

# results.append({
#     "argument_sentence": hate_, 
#     "query": query, 
#     "retireved_documents": res,

# })

# results


INFO:elastic_transport.transport:POST http://localhost:9200/news_cc/_search [status:200 duration:0.367s]


[{'argument_sentence': 'Fuck Islam and Fuck ISIS. We should repatriate all Muslims. They are all a bunch of violent criminals who do not share our values and never will. Without them Britain will be a better, safer place.',
  'query': 'fuck isis repatriate muslims',
  'retireved_documents': ['‘Assalamu Alaykom:’ peace be upon you',
   'How We Got to Here: A Charlottesville Reading List',
   'De-radicalization can work for former ISIS fighters',
   'How to Get Assyrians to Stay',
   'How to Get Christians to Stay']}]

In [27]:
# results = []
# res = [i["_source"]["document"]["title"] for i in wiki_db.search(query_=query, k=5)]

# results.append({
#     "argument_sentence": hate_, 
#     "query": query, 
#     "retireved_documents": res,

# })

# results

INFO:elastic_transport.transport:POST http://localhost:9200/knowledge/_search [status:200 duration:0.249s]


[{'argument_sentence': 'Fuck Islam and Fuck ISIS. We should repatriate all Muslims. They are all a bunch of violent criminals who do not share our values and never will. Without them Britain will be a better, safer place.',
  'query': 'fuck isis repatriate muslims',
  'retireved_documents': ['Refuting ISIS',
   'Refuting ISIS',
   'Take Off (2017 film)',
   'Terrorism in India',
   'Tareq Kamleh']}]

In [ ]:
# #queries
# query = "the executive the presidential system legislative branches partisan unproductive"
# res = search_text(wiki_ev, query_=query, k=3)
# res

[{'_index': 'knowledge',
  '_id': 'kbZa5YIBRqC1428DaQUk',
  '_score': 33.672318,
  '_ignored': ['document.text.keyword'],
  '_source': {'document': {'id': '3408791',
    'source': 'wikipedia',
    'title': 'Ruling party',
    'text': "In parliamentary systems, the majority in the legislature also controls the executive branch of government, thus leaving no possibility of opposing parties concurrently occupying the executive and legislative branches of government. In other systems, such as in an American style presidential system, the party of the president does not necessarily also have a legislative majority. A 'ruling party' is also used to describe the party of one-party states, such as the Chinese Communist Party in the People's Republic of China. In his political manifesto 'The Green Book', the late Libyan leader Muammar al-Gaddafi attacked the ability of the ruling party, using it as a basis for his opposition to partisan politics. None"}}},
 {'_index': 'knowledge',
  '_id': 'JQ3

In [ ]:
# import re

# print(", ".join(re.sub(r'\[\[(?:[^|\]]*\|)?([^\]]*)]]', "", i["_source"]["document"]["title"]).strip("[]") for i in res))

Ruling party, Fusion of powers, Student governments in the United States


In [ ]:
# ### TEST SEARCH ###

# def search_text(es, query_, k=5):
#     results = es.search(
#         index = es.elastic_index,
#         query = {
#             "match": {
#                 "document.text": query_,
#                 },
#         },
#         size=k)

#     hits = results["hits"]["hits"]
#     doc_ids = [row['_source']["document"]["id"] for row in hits]

#     return hits

# def search_topic(es, topic, k=5):
#     results = es.search(
#         index = es.elastic_index,
#         body= {
#             "size": k,
#             "query": {
#                 "match": {
#                     "document.title": topic,
#         }}})

#     hits = results["hits"]["hits"]
#     doc_ids = [row['_source']["document"]["id"] for row in hits]

#     title = hits[0]["_source"]["document"]["title"]
#     text = hits[0]["_source"]["document"]["text"]

#     return {
#         "title": title,
#         "text": text
#     }

# query = "government emails"
# text = search_text(wiki_ev, query_=query, k=10)

# print(text)


In [ ]:
### SEARCH FUNCTION ###

# TODOs: Implement as Class-DB object

# def search_text(db, query_, k=5):
#     results = db.search(
#         index = es.elastic_index,
#         query = {
#             "match": {
#                 "document.text": query_,
#                 },
#         },
#         size=k)

#     hits = results["hits"]["hits"]
#     doc_ids = [row['_source']["document"]["id"] for row in hits]

#     return hits